In [1]:
%load_ext google.cloud.bigquery

In [10]:
%%bigquery samples
SELECT 
    DEP_T, AIRLINE, ARR, DEP_DELAY, ARR_DELAY
from `going-tfx.examples.ATL_JUNE_SIGNATURE` 
where date='2006-06-12'
order by dep_t

,DEP_T,AIRLINE,ARR,DEP_DELAY,ARR_DELAY
0,545,US,PHX,4.0,-2.0
1,600,YV,IAD,15.0,18.0
2,600,EV,CVG,0.0,1.0
3,600,XE,IAH,4.0,-11.0
4,605,AA,MIA,-1.0,11.0
5,630,CO,EWR,-2.0,-3.0
6,630,F9,DEN,-2.0,-5.0
7,640,AA,DFW,-5.0,-16.0
8,645,AA,LGA,-3.0,7.0
9,645,DL,LGA,3.0,-8.0


In [11]:
samples[:2]

,DEP_T,AIRLINE,ARR,DEP_DELAY,ARR_DELAY
0,545,US,PHX,4.0,-2.0
1,600,YV,IAD,15.0,18.0


In [16]:
two_records = samples[:2].to_dict(orient='records')
two_records

[{u'AIRLINE': u'US',
  u'ARR': u'PHX',
  u'ARR_DELAY': -2.0,
  u'DEP_DELAY': 4.0,
  u'DEP_T': 545},
 {u'AIRLINE': u'YV',
  u'ARR': u'IAD',
  u'ARR_DELAY': 18.0,
  u'DEP_DELAY': 15.0,
  u'DEP_T': 600}]

In [13]:
data = samples.to_dict(orient='records')

In [14]:
len(data)

1075

In [60]:
import apache_beam as beam
from apache_beam import window
from apache_beam.options.pipeline_options import PipelineOptions

In [47]:
import time
import datetime
datetime.datetime.now()

datetime.datetime(2018, 11, 25, 21, 13, 47, 475619)

In [54]:
offset = int(time.time() // 86400 * 86400)

In [52]:
time.time()

1543180559.664374

In [56]:
class AddTimeStampDoFn(beam.DoFn):
    def process(self, element):
        timestamp = (offset + 
                      (element['DEP_T'] // 100) * 3600 +
                      (element['DEP_T'] % 100) * 60)
        time_stamped = beam.window.TimestampedValue(element, timestamp)
        yield time_stamped

In [67]:
class MakeHOD(beam.DoFn):
    def process(self, element):
        element=element.copy()
        dep_t = element['DEP_T']
        element['HOD'] = dep_t // 100
        yield element['HOD'], element

In [74]:
(
    data[:100] 
    | "Add_timestamp" >> beam.ParDo(AddTimeStampDoFn())
    | "Make_HOD" >> beam.ParDo(MakeHOD())
    | "Window_1h" >> beam.WindowInto(window.FixedWindows(3600))
    | "Group_by_HOD" >> beam.GroupByKey()
    | "Count"
)

[(8,
  [{u'AIRLINE': u'FL',
    u'ARR': u'TPA',
    u'ARR_DELAY': -9.0,
    u'DEP_DELAY': -3.0,
    u'DEP_T': 800,
    'HOD': 8},
   {u'AIRLINE': u'FL',
    u'ARR': u'RDU',
    u'ARR_DELAY': -8.0,
    u'DEP_DELAY': -6.0,
    u'DEP_T': 800,
    'HOD': 8},
   {u'AIRLINE': u'EV',
    u'ARR': u'BNA',
    u'ARR_DELAY': 9.0,
    u'DEP_DELAY': 0.0,
    u'DEP_T': 800,
    'HOD': 8},
   {u'AIRLINE': u'EV',
    u'ARR': u'MDW',
    u'ARR_DELAY': -2.0,
    u'DEP_DELAY': -4.0,
    u'DEP_T': 800,
    'HOD': 8},
   {u'AIRLINE': u'DL',
    u'ARR': u'DFW',
    u'ARR_DELAY': 33.0,
    u'DEP_DELAY': 41.0,
    u'DEP_T': 800,
    'HOD': 8},
   {u'AIRLINE': u'AA',
    u'ARR': u'DFW',
    u'ARR_DELAY': -24.0,
    u'DEP_DELAY': -2.0,
    u'DEP_T': 800,
    'HOD': 8},
   {u'AIRLINE': u'EV',
    u'ARR': u'GRR',
    u'ARR_DELAY': -2.0,
    u'DEP_DELAY': -5.0,
    u'DEP_T': 800,
    'HOD': 8},
   {u'AIRLINE': u'FL',
    u'ARR': u'PIT',
    u'ARR_DELAY': 9.0,
    u'DEP_DELAY': 3.0,
    u'DEP_T': 800,
    'HOD': 8}